In [1]:
import numpy as np

import dataset

import cv2

In [2]:
HEIGHT = 64
WIDTH = 64
DEPTH = 3

DATA_PATH = '../dataset/stl10_binary/train_X.bin'
LABEL_PATH = '../dataset/stl10_binary/train_y.bin'

In [3]:
images = dataset.read_all_images(DATA_PATH)
labels = dataset.read_labels(LABEL_PATH)

In [4]:
def get_resized_images(images):
    train_X=np.empty((images.shape[0],HEIGHT,WIDTH,3),dtype=np.uint8)
    
    for i in range(images.shape[0]):
        train_X[i] = cv2.resize(images[i],(HEIGHT,WIDTH))
    
    return train_X

In [5]:
train_X = get_resized_images(images)

In [151]:
train= train_X.reshape(5000,3,64,64)

In [44]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D,Conv2DTranspose
from keras.models import Model
from keras import backend as K
from keras.callbacks import TensorBoard


In [58]:
def get_cnn_architecture(weights_path=None):
    
    input_img = Input(shape=(64,64,3))  # adapt this if using `channels_first` image data format
    x1 = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x1)
    x2 = Conv2D(64, (3, 3), activation='relu', padding='same')(x) 
    x = MaxPooling2D((2, 2), padding='same')(x2)
    x3 = Conv2D(128, (3, 3), activation='relu', padding='same')(x) 
    x = MaxPooling2D((2, 2), padding='same')(x3)
    x4 = Conv2D(256, (3, 3), activation='relu', padding='same')(x) 
    x = MaxPooling2D((2, 2), padding='same')(x4)
    x5 = Conv2D(512, (3, 3), activation='relu', padding='same')(x) 
    
    x = UpSampling2D((2, 2))(x5)
    y1 = Conv2DTranspose(256, (3, 3), activation='relu', padding='same')(x) 
    x = UpSampling2D((2, 2))(y1)
    y2 = Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x) 
    x = UpSampling2D((2, 2))(y2)
    y3 = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x) 
    x = UpSampling2D((2, 2))(y3)
    y4 = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x) 
    y5 = Conv2DTranspose(3, (3, 3), activation='relu', padding='same')(y4) 
    
    layers = y5
    
    model = Model(input_img,layers)
    print model.summary()
    return model

In [59]:
model = get_cnn_architecture()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_133 (Conv2D)          (None, 64, 64, 64)        1792      
_________________________________________________________________
max_pooling2d_72 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_134 (Conv2D)          (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_73 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_135 (Conv2D)          (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_74 (MaxPooling (None, 8, 8, 128)         0         
__________

In [6]:
model.compile(optimizer='sgd', loss = 'mean_squared_error', metrics = ['accuracy'])

In [159]:
labels

array([2, 6, 2, ..., 2, 8, 6], dtype=uint8)

In [160]:
model.fit(train,labels,batch_size=32,nb_epoch=10,verbose=1,shuffle=True)

Epoch 1/10
5000/5000 [==============================] - 28s - loss: 28.5000 - acc: 0.1000    
Epoch 2/10
5000/5000 [==============================] - 26s - loss: 28.5000 - acc: 0.1000    
Epoch 3/10
5000/5000 [==============================] - 27s - loss: 28.5000 - acc: 0.1000    
Epoch 4/10
5000/5000 [==============================] - 27s - loss: 28.5000 - acc: 0.1000    
Epoch 5/10
5000/5000 [==============================] - 25s - loss: 28.5000 - acc: 0.1000    
Epoch 6/10
5000/5000 [==============================] - 25s - loss: 28.5000 - acc: 0.1000    
Epoch 7/10
5000/5000 [==============================] - 26s - loss: 28.5000 - acc: 0.1000    
Epoch 8/10
5000/5000 [==============================] - 26s - loss: 28.5000 - acc: 0.1000    
Epoch 9/10
5000/5000 [==============================] - 26s - loss: 28.5000 - acc: 0.1000    
Epoch 10/10
5000/5000 [==============================] - 26s - loss: 28.5000 - acc: 0.1000    


In [175]:
model.predict()

TypeError: Error when checking : data should be a Numpy array, or list/dict of Numpy arrays. Found: (None, array([[[146, 143, 110, ..., 123,  85, 136],
        [121,  82, 134, ...,  87, 141, 125],
        [ 89, 142, 126, ..., 162, 159, 126],
        ..., 
        [ 99,  61, 116, ...,  78, 119, 106],...